In [ ]:
#Installing my libraries 
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
import warnings
from collections import Counter
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Dense


In [ ]:
df = pd.read_csv('bankloan.csv')

In [ ]:
df.head()

In [ ]:
df['LoanAmount'].isna().sum()# because i noticed there was NaN in the LoanAmount column

In [ ]:
df.isna().sum()

In [ ]:
df.shape

since we have 614 rows.. then removing just a couple 50 rows should be okay.. because we want to get clean data right??

In [ ]:
df = df.dropna()
df.isna().any()

We have successfully cleared all our NaN values
Let's check our shape now

In [ ]:
df.shape

We dropped a number of rows.. but that should not harm our models as much

In [ ]:
df.head()

In [ ]:
df = df.drop('Loan_ID', axis=1) #We dont need IDs when building our models
df['LoanAmount']=(df['LoanAmount']*1000).astype(int)
Counter(df['Loan_Status']) #We want to know if our dataset is imbalanced

In [ ]:
pre_y=df['Loan_Status']
pre_X=df.drop('Loan_Status', axis=1)


In [ ]:
pre_X

In [ ]:
dm_X = pd.get_dummies(pre_X).astype(int) #We are using OneHotEncoding to work on the data so that the computer knows that it will work with numbers and not strings
dm_X

In [ ]:
dm_y = pre_y.map(dict(Y=1, N=0))#we are not using get dummies here because we want the values to be on one column
dm_y

In [ ]:
smote = SMOTE(sampling_strategy='minority')#we are using SMOTE as our sampling method to make the stuff balanced
X1, y = smote.fit_resample(dm_X, dm_y)
sc = MinMaxScaler()
X = sc.fit_transform(X1)

In [ ]:
Counter(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
classifier = Sequential()
classifier.add(Dense(200, activation='relu', kernel_initializer='random_normal', input_dim=X_test.shape[1]))
classifier.add(Dense(400, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size=20, epochs=100, verbose=0)
eval_model = classifier.evaluate(X_train, y_train)
eval_model

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap='Reds', cbar=True); #annot=True to annotate cells

#labels, title and ticks
ax.set_xlabel('Predicted');
ax.set_ylabel('Actual');
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['No', 'Yes']); 
ax.yaxis.set_ticklabels(['No', 'Yes']); 
plt.show()

In [ ]:
cm #for some reason the other numbers are not showing

In [ ]:
import pickle
import joblib
filename = 'loan_model.pkl'
joblib.dump(classifier, filename)

In [ ]:
X_test

In [ ]:
test_df = pd.DataFrame(X_test, columns=dm_X.columns)

In [ ]:
test_df

In [ ]:
test_df.to_excel('test.xlsx', index=False)

In [ ]:
with open('scalers.pkl', 'rb') as f:
    scaler = joblib.load(f)